# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,naranjal,-2.6667,-79.6167,24.74,84,100,1.97,EC,2023-11-20
1,1,bubaque,11.2833,-15.8333,28.05,74,100,4.91,GW,2023-11-20
2,2,whitehorse,60.7161,-135.0538,-15.55,84,100,1.54,CA,2023-11-20
3,3,port mathurin,-19.6833,63.4167,25.25,80,85,7.31,MU,2023-11-20
4,4,grytviken,-54.2811,-36.5092,1.19,99,100,2.71,GS,2023-11-20


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    color='City',
    size='Humidity',
    xlabel='x',
    ylabel='y',
    hover_cols=['City', 'Humidity']
)


# Display the map
map_plot



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
criteria = (city_data_df['Cloudiness'] == 0) & (city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27) & (city_data_df['Wind Speed'] < 4.5) 

filtered_cities_df = city_data_df[criteria]


# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,34,melchor de mencos,17.0686,-89.1522,26.00,89,0,0.00,GT,2023-11-20
105,105,santa ana,33.7456,-117.8678,22.82,16,0,2.68,US,2023-11-20
262,262,carnarvon,-24.8667,113.6333,25.39,67,0,2.69,AU,2023-11-20
308,308,mae hong son,19.3003,97.9685,23.13,78,0,0.46,TH,2023-11-20
338,338,rosarito,32.3333,-117.0333,22.59,46,0,1.65,MX,2023-11-20
354,354,touros,-5.1989,-35.4608,26.40,78,0,4.40,BR,2023-11-20
364,364,motueka,-41.1333,173.0167,22.83,40,0,3.22,NZ,2023-11-20
390,390,alice,27.7523,-98.0697,23.69,83,0,2.57,US,2023-11-20
403,403,trairi,-3.2778,-39.2689,26.74,77,0,4.42,BR,2023-11-20
579,579,dzilam gonzalez,21.2833,-88.9333,25.79,72,0,3.97,MX,2023-11-20


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
34,melchor de mencos,GT,17.0686,-89.1522,89,
105,santa ana,US,33.7456,-117.8678,16,
262,carnarvon,AU,-24.8667,113.6333,67,
308,mae hong son,TH,19.3003,97.9685,78,
338,rosarito,MX,32.3333,-117.0333,46,
354,touros,BR,-5.1989,-35.4608,78,
364,motueka,NZ,-41.1333,173.0167,40,
390,alice,US,27.7523,-98.0697,83,
403,trairi,BR,-3.2778,-39.2689,77,
579,dzilam gonzalez,MX,21.2833,-88.9333,72,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000  # 10km


params = {
    "radius": radius,
    "categories": "accommodation.hotel",
    "limit": 1,  # Limit to one result per city
    "apiKey": geoapify_key,  # Assuming you have the API key defined
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    print(requests.get(base_url, params=params))
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
<Response [200]>
melchor de mencos - nearest hotel: Hotel Central
<Response [200]>
santa ana - nearest hotel: Holiday Inn Express
<Response [200]>
carnarvon - nearest hotel: No hotel found
<Response [200]>
mae hong son - nearest hotel: Jongkham Place
<Response [200]>
rosarito - nearest hotel: WorldMark La Paloma
<Response [200]>
touros - nearest hotel: Pousada Atlântico
<Response [200]>
motueka - nearest hotel: Hotel Motueka
<Response [200]>
alice - nearest hotel: No hotel found
<Response [200]>
trairi - nearest hotel: Pousada Villa Aurora
<Response [200]>
dzilam gonzalez - nearest hotel: No hotel found
<Response [200]>
nouna - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
34,melchor de mencos,GT,17.0686,-89.1522,89,Hotel Central
105,santa ana,US,33.7456,-117.8678,16,Holiday Inn Express
262,carnarvon,AU,-24.8667,113.6333,67,No hotel found
308,mae hong son,TH,19.3003,97.9685,78,Jongkham Place
338,rosarito,MX,32.3333,-117.0333,46,WorldMark La Paloma
354,touros,BR,-5.1989,-35.4608,78,Pousada Atlântico
364,motueka,NZ,-41.1333,173.0167,40,Hotel Motueka
390,alice,US,27.7523,-98.0697,83,No hotel found
403,trairi,BR,-3.2778,-39.2689,77,Pousada Villa Aurora
579,dzilam gonzalez,MX,21.2833,-88.9333,72,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    color='City',
    xlabel='x',
    ylabel='y',
    hover_cols=['City', 'Humidity', 'Hotel Name', 'Country']

)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)